In [3]:
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_md")
# doc = nlp("The shaft is positioned through support bearings. A magnetic off-loader provides a magnetic force to move the shaft axially in regard to the bearings.")
# for token in doc:
#     print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
#             token.shape_, token.is_alpha, token.is_stop)

print ("-------") 
doc = nlp("the magnetic force provided levitates the shaft")
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)
    
displacy.serve(doc, style="dep")

-------
the the DET DT det xxx True True
magnetic magnetic ADJ JJ amod xxxx True False
force force NOUN NN nsubj xxxx True False
provided provide VERB VBN acl xxxx True False
levitates levitate VERB VBZ ROOT xxxx True False
the the DET DT det xxx True True
shaft shaft NOUN NN dobj xxxx True False



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [ ]:

doc2 = nlp("a bearingless hub assembly comprises a rim to receive a tube magnet")
displacy.serve(doc2, style="dep")




Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...



In [6]:

from spacy import displacy
displacy.serve(doc,style='dep')
def get_pps(doc):
    "Function to get PPs from a parsed document."
    pps = []
    for token in doc:
        # Try this with other parts of speech for different subtrees.
        if token.tag_ == 'VBZ':
            pp = ' '.join([tok.orth_ for tok in token.subtree])
            pps.append(pp)
    return pps
import spacy

nlp = spacy.load('en_core_web_md')
ex= '"The invention relates to a magnetic bearing for generating magnetic forces."'
doc = nlp(ex)

print(get_pps(doc))

['" The invention relates to a magnetic bearing for generating magnetic forces . "']


In [7]:
import spacy
from tqdm import tqdm
import re
from spacy.matcher import PhraseMatcher

nlp = spacy.load("en_core_web_md")
matcher1 = PhraseMatcher(nlp.vocab,attr='POS')

terms = ['relate to','positioned through','engageable with','so as to','makes it possible to']

# 'in regard to' ,'in response to', 'make it possible to'

# 注意：只有使用 nlp.make_doc 才能加速
patterns=[nlp(term) for term in terms]
matcher1.add('VerbPhrase',patterns)

# matcher.add("TerminologyList", [nlp('relate to'),nlp('positioned through')])



# with open ('examples/patent_example.txt','r') as f:
#     num_file = sum([1 for i in open('examples/patent_example.txt', "r")])
#     for idx, line in enumerate(tqdm (f,total=num_file)):
#         print ("line: %s " % (str(idx)))
#         sentence = re.sub(' +', ' ',line.strip())




#         doc = nlp(sentence)
#         matches = matcher1(doc)
#         for match_id, start, end in matches:

#             span = doc[start:end]


In [8]:
import en_core_web_md
from process import parse_sentence
from tqdm import tqdm
from transformers import AutoTokenizer, BertModel, GPT2Model
from mapper import Map, deduplication
nlp = en_core_web_md.load()
import torch
import numpy as np
from utils import compress_attention, create_mapping, BFS, build_graph, is_word
from multiprocessing import Pool
import spacy
import en_core_web_md
import torch
from transformers import AutoTokenizer, BertModel, GPT2Model
from constant import invalid_relations_set
from utils import compress_attention,build_graph
import re

language_model = 'allenai/scibert_scivocab_uncased'
tokenizer = AutoTokenizer.from_pretrained(language_model)
encoder = BertModel.from_pretrained(language_model)
encoder.eval()
use_cuda =False
threshold = 0.01
# def process_matrix(attentions, layer_idx = -1, head_num = 0, avg_head=False, trim=True, use_cuda=True):
#     if avg_head:
#         if use_cuda:
#             attn =  torch.mean(attentions[0][layer_idx], 0).cpu()
#         else:
#             attn = torch.mean(attentions[0][layer_idx], 0)
#         attention_matrix = attn.detach().numpy()
#     else:
#         attn = attentions[0][layer_idx][head_num]
#         if use_cuda:
#             attn = attn.cpu()
#         attention_matrix = attn.detach().numpy()

#     attention_matrix = attention_matrix[1:-1, 1:-1]

#     return attention_matrix




with open ('examples/patent_example.txt','r') as f:
    num_file = sum([1 for i in open('examples/patent_example_1.txt', "r")])
    #tqdm summmarization
    for idx, line in enumerate(tqdm (f,total=num_file)):

        ##modification starts below###
#         sentence= line.strip()
        sentence = re.sub(' +', ' ',line.strip())
        
        ##modification ends here\####
#         if len(sentence):
#             valid_triplets = []
#             for sent in nlp(sentence).sents:
#                 sentence= sent.text
#                 tokenizer_name = str(tokenizer.__str__)
#                 inputs, tokenid2word_mapping, token2id, noun_chunks  = create_mapping(sentence, return_pt=True, nlp=nlp, tokenizer=tokenizer)

#                 with torch.no_grad():
#                     if use_cuda:
#                         for key in inputs.keys():
#                             inputs[key] = inputs[key].cuda()
#                     outputs = encoder(**inputs, output_attentions=True)
#                 trim = True
#                 if 'GPT2' in tokenizer_name:
#                     trim  = False

#                 '''
#                 Use average of last layer attention : page 6, section 3.1.2
#                 '''
#                 attention = process_matrix(outputs[2], avg_head=True, trim=trim, use_cuda=use_cuda)

#                 merged_attention = compress_attention(attention, tokenid2word_mapping)
#                 attn_graph = build_graph(merged_attention)
#             print ('attention:{}'.format(attention))
#             print ('merged_attention:{}'.format(merged_attention))
#             print ("attn_graph:{}".format(attn_graph))
#             print ("attention_shape:{0},merged_attention_shape:{1},attention_graph_shape:{2}".\
#                    format(attention.shape,              
#                           merged_attention.shape,
#                          len(attn_graph)))

        valid_triplets = []
        count = 0
        doc = nlp(sentence)
        print ("count:{}".format(count))
        
        ###test for verb phrases start###
        matches = matcher1(doc)
        verb_phrase_start_chunk = []
        verb_phrase_end_chunk = []
        verb_phrase_chunk=[]
        print (matches)
        for match_id, start, end in matches:
            span = doc[start:end]
            verb_phrase_start_chunk.append(start)
            verb_phrase_end_chunk.append(end)
            verb_phrase_chunk.append(span.text)
#             print("start: %s, end: %s text :%s" % (start,end,span.text))
            
        ### test for verb phrase ends####
        
        
        
#         for sent in nlp(sentence).sents:
#             print ("sent:{}".format(sent))
#             print ("sent start :%s sent end :%s"% (sent.start, sent.end))
            
        start_chunk = []
        end_chunk = []
        noun_chunks = []

        for chunk in doc.noun_chunks:
            noun_chunks.append(chunk.text)
            start_chunk.append(chunk.start)
            end_chunk.append(chunk.end)
            
#         start_chunk= sorted(start_chunk + verb_phrase_start_chunk)
#         end_chunk = sorted (end_chunk +verb_phrase_end_chunk)
        chunk_id = 0
        token2id ={ }
        mode = 1
        sentence_mapping = []

        #### modification starts below,
        ### token2id [sentence_mapping[-1]]= len(token2id) --->token2id [sentence_mapping[-1]]= len(sentence_mapping)-1
        ### because token2id is a dictionary, when you len it, it will automatically remove the duplicated keys, this will
        ### cause problems when the words occurs more than once

            
        dic = dict (zip (start_chunk+verb_phrase_start_chunk,noun_chunks+verb_phrase_chunk))
        
        
        dic = {k: v for k, v in sorted(dic.items(), key=lambda dic:dic[0],reverse= False)}

 
        start_chunk= list(dic.keys())
        chunks = list(dic.values())
        end_chunk = sorted(end_chunk+ verb_phrase_end_chunk)
        print ('start_chunk: %s, end_chunk: %s'%(str(start_chunk),str(end_chunk)))
        print (dic)
        for idx,token in enumerate (doc):
            if idx in start_chunk:
                

                mode= 1
                sentence_mapping.append (chunks[chunk_id])
                token2id [sentence_mapping[-1]]= len(sentence_mapping)-1
#                 token2id [sentence_mapping[-1]]= len(token2id)
                chunk_id +=1 
                continue
            if idx in end_chunk:
                mode = 0

                
            if mode == 0:
                sentence_mapping.append (token.text)
                token2id [sentence_mapping[-1]]= len(sentence_mapping)-1
#                 token2id [sentence_mapping[-1]]= len(token2id)

        ### modication ends here 
                
        print ('sentence_mapping:{}'.format (sentence_mapping))

        token_ids = []
        tokenid2word_mapping = []
        ###modification starts below, adding id2token
        id2token = {key:value for key,value in enumerate(sentence_mapping)}
        ###modification ends here
        for token in sentence_mapping:
            subtoken_ids = tokenizer(str(token), add_special_tokens=False)['input_ids']
            tokenid2word_mapping += [ token2id[token] ]*len(subtoken_ids)
            token_ids += subtoken_ids
        print (id2token)


4it [00:01,  1.25it/s]                       

count:0
[(10639893602612371815, 2, 4), (10639893602612371815, 28, 30), (10639893602612371815, 72, 74), (10639893602612371815, 97, 99)]
start_chunk: [0, 2, 4, 9, 12, 18, 23, 26, 28, 30, 34, 39, 43, 47, 51, 55, 60, 64, 68, 72, 74, 78, 83, 87, 90, 94, 97, 99, 102, 105, 108, 112, 117, 123], end_chunk: [2, 4, 8, 11, 17, 22, 25, 27, 30, 33, 38, 42, 45, 50, 54, 58, 63, 67, 71, 74, 77, 81, 85, 89, 93, 96, 99, 101, 103, 107, 111, 115, 122, 125]
{0: 'The invention', 2: 'relates to', 4: 'a roadable aircraft vehicle', 9: 'related systems', 12: 'An example roadable aircraft vehicle', 18: 'a vehicle drive system', 23: 'an engine', 26: 'gearbox', 28: 'engageable with', 30: 'an automotive driveline', 34: 'at least one propeller', 39: 'a user interface', 43: 'a display', 47: 'the drive system', 51: 'an automotive mode', 55: 'a steering wheel', 60: 'a flight mode', 64: 'a control stick', 68: 'a control system', 72: 'switching between', 74: 'the flight mode', 78: 'the automotive mode', 83: 'a system', 87

13it [00:01,  2.47it/s]

{0: 'An improved flywheel system', 1: 'for', 2: 'storing', 3: 'energy', 4: 'and', 5: 'providing', 6: 'the stored energy', 7: 'comprising', 8: ':', 9: 'wherein', 10: 'the improvement', 11: 'comprises', 12: 'a feedback control system', 13: 'comprising', 14: 'a sensor', 15: 'mounted in', 16: 'the upper bearing housing', 17: ',', 18: 'said', 19: 'sensor', 20: 'measuring', 21: 'the distance', 22: 'of', 23: 'a gap', 24: 'between', 25: 'a top end', 26: 'of', 27: 'the upper shaft', 28: 'and', 29: 'a lower surface', 30: 'of', 31: 'the sensor', 32: ',', 33: 'the sensor', 34: 'generating', 35: 'an electrical signal', 36: 'that', 37: 'changes', 38: 'in', 39: 'value', 40: 'in', 41: 'relationship', 42: 'to', 43: 'said', 44: 'distance', 45: ',', 46: 'the electrical signal', 47: 'from', 48: 'the sensor', 49: 'processed by', 50: 'a controller', 51: ',', 52: 'said', 53: 'controller', 54: 'provides', 55: 'variable electric current', 56: 'to', 57: 'the electro-magnet', 58: 'within', 59: 'the magnetic off-

24it [00:01,  4.79it/s]

count:0
[]
start_chunk: [], end_chunk: []
{}
sentence_mapping:[]
{}
count:0
[(10639893602612371815, 17, 19)]
start_chunk: [2, 7, 11, 17, 19, 22], end_chunk: [6, 8, 13, 19, 21, 24]
{2: 'The improved flywheel system', 7: 'claim', 11: 'a stator', 17: 'converted between', 19: 'parallel windings', 22: 'series windings'}
sentence_mapping:['The improved flywheel system', 'of', 'claim', '1', 'further', 'including', 'a stator', 'winding', 'configured', 'to', 'be', 'converted between', 'parallel windings', 'and', 'series windings', '.']
{0: 'The improved flywheel system', 1: 'of', 2: 'claim', 3: '1', 4: 'further', 5: 'including', 6: 'a stator', 7: 'winding', 8: 'configured', 9: 'to', 10: 'be', 11: 'converted between', 12: 'parallel windings', 13: 'and', 14: 'series windings', 15: '.'}
count:0
[]
start_chunk: [], end_chunk: []
{}
sentence_mapping:[]
{}
count:0
[]
start_chunk: [2, 7, 10, 14, 19, 23, 29], end_chunk: [6, 8, 13, 18, 22, 27, 33]
{2: 'The improved flywheel system', 7: 'claim', 10: 'the

32it [00:01, 16.64it/s]

count:0
[]
start_chunk: [0, 1, 7, 12, 17, 21, 23], end_chunk: [1, 6, 9, 15, 19, 22, 28]
{0: 'd.', 1: 'a magnetic off-loader', 7: 'an electromagnet', 12: 'a magnetic force', 17: 'the rotor', 21: 'regard', 23: 'the upper and lower bearings'}
sentence_mapping:['d.', 'a magnetic off-loader', 'including', 'an electromagnet', 'configured', 'to', 'provide', 'a magnetic force', 'to', 'move', 'the rotor', 'axially', 'in', 'regard', 'to', 'the upper and lower bearings', ',', 'and']
{0: 'd.', 1: 'a magnetic off-loader', 2: 'including', 3: 'an electromagnet', 4: 'configured', 5: 'to', 6: 'provide', 7: 'a magnetic force', 8: 'to', 9: 'move', 10: 'the rotor', 11: 'axially', 12: 'in', 13: 'regard', 14: 'to', 15: 'the upper and lower bearings', 16: ',', 17: 'and'}
count:0
[(10639893602612371815, 9, 11), (10639893602612371815, 64, 66), (10639893602612371815, 91, 93)]
start_chunk: [1, 6, 7, 9, 11, 17, 19, 22, 25, 29, 33, 37, 40, 43, 49, 51, 54, 58, 62, 64, 66, 70, 72, 76, 79, 88, 91, 93, 99], end_chunk: